Quick experiment to see which is better at detecting truthful answers

- model outputs
- hs
- supressed activations (Hypothesis this is better)

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
from einops import rearrange, repeat
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.data import DataCollatorForLanguageModeling

import torch
from torch import Tensor
from torch.nn.functional import (
    binary_cross_entropy_with_logits as bce_with_logits,
)
from torch.nn.functional import (
    cross_entropy,
)

from jaxtyping import Float
from torch import Tensor

from activation_store.collect import activation_store, default_postprocess_result

## Load model

In [4]:
model_name = "Qwen/Qwen2.5-3B-Instruct"
# model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="eager",  # flex_attention  flash_attention_2 sdpa eager
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.paddding_side = "left"
tokenizer.truncation_side = "left"

Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Load data and tokenize

In [5]:
# N = 316
max_length = 64
split = "train"
ds1 = load_dataset("Yik/truthfulQA-bool", split=split, keep_in_memory=False)

sys_msg = """You will be given a statement, predict if it is true according to wikipedia, and return only 0 for false and 1 for true.
"""


def proc(row):
    messages = [
        {"role": "system", "content": sys_msg},
        {"role": "user", "content": row["question"]},
    ]
    return tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_dict=True,
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )


ds2 = ds1.map(proc).with_format("torch")
new_cols = list(set(ds2.column_names) - set(ds1.column_names)) + ["label"]
ds2 = ds2.select_columns(new_cols)
ds2

Dataset({
    features: ['attention_mask', 'input_ids', 'label'],
    num_rows: 316
})

In [6]:
# how many rows are truncated (and therefore have an attention mask of all 1s)
ds2['attention_mask'].float().all(1).float().mean()

tensor(0.1709)

## Data loader

In [7]:
collate_fn = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
ds = DataLoader(ds2, batch_size=1, collate_fn=collate_fn)
print(ds)


## Collect activations

In [ ]:
# choose layers to cache
layer_groups = {
    'mlp.down_proj': [k for k,v in model.named_modules() if k.endswith('mlp.down_proj')][10:25],
    'self_attn': [k for k,v in model.named_modules() if k.endswith('.self_attn')][10:25],
    'mlp.up_proj': [k for k,v in model.named_modules() if k.endswith('mlp.up_proj')][10:25],
}
layer_groups = []

: 

In [ ]:
f = activation_store(ds, model, layers=layer_groups)
f

In [ ]:
ds_a = Dataset.from_parquet(str(f), split=split).with_format("torch")
ds_a

In [ ]:
max_length

In [ ]:
{k:v.shape for k,v in ds_a[0].items()}

In [ ]:
# sanity test generate
b = next(iter(ds))
b = {k: v.to(model.device) for k, v in b.items()}
o = model.generate(
    inputs=b["input_ids"],
    attention_mask=b["attention_mask"],
    max_new_tokens=3,
)
gent = tokenizer.batch_decode(o, skip_special_tokens=False)
for g in gent:
    print(g)
    print("---")

## Get supressed activations

In [ ]:
@torch.no_grad()
def get_supressed_activations(
    hs: Float[Tensor, "l b t h"], w_out, w_inv
) -> Float[Tensor, "l b t h"]:
    """
    Novel experiment: Here we define a transform to isolate supressed activations, where we hypothesis that style/concepts/scratchpads and other internal only representations must be stored.

    See the following references for more information:

    - https://arxiv.org/pdf/2401.12181
        - > Suppression neurons that are similar, except decrease the probability of a group of related tokens
        - > We find a striking pattern which is remarkably consistent across the different seeds: after about the halfway point in the model, prediction neurons become increasingly prevalent until the very end of the network where there is a sudden shift towards a much larger number of suppression neurons.

    - https://arxiv.org/html/2406.19384
        - > Previous work suggests that networks contain ensembles of “prediction" neurons, which act as probability promoters [66, 24, 32] and work in tandem with suppression neurons (Section 5.4).


    Output:
    - supression amount: This is a tensor of the same shape as the input hs, where the values are the amount of suppression that occured at that layer, and the sign indicates if it was supressed or promoted. How do we calulate this? We project the hs using the output_projection, look at the diff from the last layer, and then project it back using the inverse of the output projection. This gives us the amount of suppression that occured at that layer.
    """
    hs_flat = rearrange(hs[:, :, -1:], "l b t h -> (l b t) h")
    hs_out_flat = torch.nn.functional.linear(hs_flat, w_out)
    hs_out = rearrange(
        hs_out_flat, "(l b t) h -> l b t h", l=hs.shape[0], b=hs.shape[1], t=1
    )
    diffs = hs_out[:, :, :].diff(dim=0)
    diffs_flat = rearrange(diffs, "l b t h -> (l b t) h")
    # W_inv = get_cache_inv(w_out)

    diffs_inv_flat = torch.nn.functional.linear(diffs_flat.to(dtype=w_inv.dtype), w_inv)
    diffs_inv = rearrange(
        diffs_inv_flat, "(l b t) h -> l b t h", l=hs.shape[0] - 1, b=hs.shape[1], t=1
    ).to(w_out.dtype)

    # add on missing first layer
    torch.zeros_like(diffs_inv[:1]).to(hs.device)
    diffs_inv = torch.cat(
        [torch.zeros_like(diffs_inv[:1]).to(hs.device), diffs_inv], dim=0
    )
    return diffs_inv

In [ ]:
def get_uniq_token_ids(tokens):
    token_ids = tokenizer(
        tokens, return_tensors="pt", add_special_tokens=False, padding=True
    ).input_ids
    token_ids = torch.tensor(list(set([x[0] for x in token_ids]))).long()
    print("before", tokens)
    print("after", tokenizer.batch_decode(token_ids))
    return token_ids


false_tokens = ["0", "0 ", "0\n", "false", "False "]
false_token_ids = get_uniq_token_ids(false_tokens)

true_tokens = ["1", "1 ", "1\n", "true", "True "]
true_token_ids = get_uniq_token_ids(true_tokens)

In [ ]:
# now we map to 1) calc supressed activations 2) llm answer (prob of 0 vs prob of 1)

Wo = model.get_output_embeddings().weight.detach().clone().cpu()
Wo_inv = torch.pinverse(Wo.clone().float())


def proc(o):
    # TODO batch it
    """Process model outputs"""

    # get llm ans
    log_probs = o["logits"][-1].log_softmax(0)
    false_log_prob = log_probs.index_select(0, false_token_ids).sum()
    true_log_prob = log_probs.index_select(0, true_token_ids).sum()
    o["llm_ans"] = torch.stack([false_log_prob, true_log_prob])
    o["llm_log_prob_true"] = true_log_prob - false_log_prob

    # get supressed activations
    hs = o["hidden_states"][None]
    hs = rearrange(hs, "b l t h -> l b t h")
    diffs_inv = get_supressed_activations(hs, Wo.to(hs.dtype), Wo_inv.to(hs.dtype))

    # we will only take the last half of layers, and the last token
    layer_half = hs.shape[0] // 2
    
    hs = rearrange(hs, "l b t h -> b l t h").squeeze(0)[layer_half:-2]
    diffs_inv = rearrange(diffs_inv, "l b t h -> b l t h").squeeze(0)[layer_half:-2]

    o["hidden_states"] = hs.half()
    o["diffs_inv"] = diffs_inv.half()
    return o


ds_a2 = ds_a.map(proc, writer_batch_size=1, num_proc=None)
ds_a2

In [ ]:
{k: v.shape for k,v in ds_a2[0].items() if isinstance(v, torch.Tensor)}


In [ ]:
# # # now convert diffs_inv to supressed_mask and hs_sup

# def proc2(o, eps = 1.0e-2):
#     diffs_inv = o["diffs_inv"]
#     hs = o["hidden_states"] # [b l h]
#     supressed_mask = (diffs_inv < -eps).to(hs.dtype)# [b l h]

#     o['hs_sup'] = hs * supressed_mask
#     o['supressed_mask'] = supressed_mask
#     return o

# ds_a2 = ds_a2.map(proc2, writer_batch_size=64, num_proc=None, batched=True, batch_size=64)
# ds_a2

## Predict

In [ ]:
# https://github.com/EleutherAI/ccs/blob/8a4bf687712cc03ef72973c8235944566d59053b/ccs/training/supervised.py#L9





class Classifier(torch.nn.Module):
    """Linear classifier trained with supervised learning."""

    def __init__(
        self,
        input_dim: int,
        num_classes: int = 2,
        device: str | torch.device | None = None,
        dtype: torch.dtype | None = None,
    ):
        super().__init__()

        self.linear = torch.nn.Linear(
            input_dim, num_classes if num_classes > 2 else 1, device=device, dtype=dtype
        )
        self.linear.bias.data.zero_()
        # self.linear.weight.data.zero_()

    def forward(self, x: Tensor) -> Tensor:
        return self.linear(x).squeeze(-1)

    @torch.enable_grad()
    def fit(
        self,
        x: Tensor,
        y: Tensor,
        *,
        l2_penalty: float = 0.001,
        max_iter: int = 10_000,
    ) -> float:
        """Fits the model to the input data using L-BFGS with L2 regularization.

        Args:
            x: Input tensor of shape (N, D), where N is the number of samples and D is
                the input dimension.
            y: Target tensor of shape (N,) for binary classification or (N, C) for
                multiclass classification, where C is the number of classes.
            l2_penalty: L2 regularization strength.
            max_iter: Maximum number of iterations for the L-BFGS optimizer.

        Returns:
            Final value of the loss function after optimization.
        """
        optimizer = torch.optim.LBFGS(
            self.parameters(),
            line_search_fn="strong_wolfe",
            max_iter=max_iter,
        )

        num_classes = self.linear.out_features
        loss_fn = bce_with_logits if num_classes == 1 else cross_entropy
        loss = torch.inf
        y = y.to(
            torch.get_default_dtype() if num_classes == 1 else torch.long,
        )

        def closure():
            nonlocal loss
            optimizer.zero_grad()

            # Calculate the loss function
            logits = self(x).squeeze(-1)
            loss = loss_fn(logits, y)
            if l2_penalty:
                reg_loss = loss + l2_penalty * self.linear.weight.square().sum()
            else:
                reg_loss = loss

            reg_loss.backward()
            return float(reg_loss)

        optimizer.step(closure)
        return float(loss)


In [ ]:
# first try llm


def roc_auc(y_true: Tensor, y_pred: Tensor) -> Tensor:
    """Area under the receiver operating characteristic curve (ROC AUC).

    Unlike scikit-learn's implementation, this function supports batched inputs of
    shape `(N, n)` where `N` is the number of datasets and `n` is the number of samples
    within each dataset. This is primarily useful for efficiently computing bootstrap
    confidence intervals.

    Args:
        y_true: Ground truth tensor of shape `(N,)` or `(N, n)`.
        y_pred: Predicted class tensor of shape `(N,)` or `(N, n)`.

    Returns:
        Tensor: If the inputs are 1D, a scalar containing the ROC AUC. If they're 2D,
            a tensor of shape (N,) containing the ROC AUC for each dataset.
    """
    if y_true.shape != y_pred.shape:
        raise ValueError(
            f"y_true and y_pred should have the same shape; "
            f"got {y_true.shape} and {y_pred.shape}"
        )
    if y_true.dim() not in (1, 2):
        raise ValueError("y_true and y_pred should be 1D or 2D tensors")

    # Sort y_pred in descending order and get indices
    indices = y_pred.argsort(descending=True, dim=-1)

    # Reorder y_true based on sorted y_pred indices
    y_true_sorted = y_true.gather(-1, indices)

    # Calculate number of positive and negative samples
    num_positives = y_true.sum(dim=-1)
    num_negatives = y_true.shape[-1] - num_positives

    # Calculate cumulative sum of true positive counts (TPs)
    tps = torch.cumsum(y_true_sorted, dim=-1)

    # Calculate cumulative sum of false positive counts (FPs)
    fps = torch.cumsum(1 - y_true_sorted, dim=-1)

    # Calculate true positive rate (TPR) and false positive rate (FPR)
    tpr = tps / num_positives.view(-1, 1)
    fpr = fps / num_negatives.view(-1, 1)

    # Calculate differences between consecutive FPR values (widths of trapezoids)
    fpr_diffs = torch.cat(
        [fpr[..., 1:] - fpr[..., :-1], torch.zeros_like(fpr[..., :1])], dim=-1
    )

    # Calculate area under the ROC curve for each dataset using trapezoidal rule
    return torch.sum(tpr * fpr_diffs, dim=-1).squeeze()


### Score llm output

In [ ]:
train_test_split = 200
a, b = ds_a2["llm_log_prob_true"] > 0, ds_a2["label"]
score = roc_auc(b[train_test_split:], a[train_test_split:])
print(f"LLM score: {score:.2f} roc auc, n={len(a[train_test_split:])}")

### score hidden states and activations

In [ ]:
def train_linear_prob_on_dataset(
    X,
    name="",
    device: str = "cuda",
):
    print(X.shape)
    X = X.view(len(X), -1).to(device)

    # norm X
    X = (X - X.mean()) / X.std()
    y = ds_a2["label"].to(device)
    X_train, y_train = X[:train_test_split], y[:train_test_split]
    X_test, y_test = X[train_test_split:], y[train_test_split:]
    # data.shape
    lr_model = Classifier(X.shape[-1], device=device)
    lr_model.fit(X_train, y_train)

    y_pred = lr_model.forward(X_test)

    score = roc_auc(y_test, y_pred)
    print(f"score for probe({name}): {score:.3f} roc auc, n={len(X_test)}")
    return score.cpu().item()

In [ ]:
def calc_supp_thresh(hs, diffs_inv, eps = 1.0e-2):
    supressed_mask = (diffs_inv < -eps).to(hs.dtype)
    hs_sup = hs * supressed_mask
    return hs_sup, supressed_mask

In [ ]:
reductions = {
    "mean": lambda x: x.mean(0),
    "max": lambda x: x.max(0)[0],
    "sum": lambda x: x.sum(0),
    "last": lambda x: x[-1],
    "first": lambda x: x[0],
    "none": lambda x: x,
}
results = []

# first try hidden states
for r1 in reductions:
    for dn in [ "hidden_states",'mlp.down_proj',
 'self_attn',
 'mlp.up_proj',]:
        r1f = reductions[r1]
        try:
            X = torch.stack([r1f(x) for x in ds_a2[dn]])
            name = f"{dn} {r1}"
            score = train_linear_prob_on_dataset(X, name)
            results.append((name, score))
        except Exception as e:
            print(f"error with {name}")
            print(e)

### score supressed activations

In [ ]:
def calc_hs_sup(o, eps = 1.0e-2):
    diffs_inv = o["diffs_inv"]
    hs = o["hidden_states"] # [b l h]
    if eps > 0:
        supressed_mask = (diffs_inv > eps).to(hs.dtype)# [b l h]
    else:
        supressed_mask = (diffs_inv < eps).to(hs.dtype)

    o['supressed_hs'] = hs * supressed_mask
    o['supressed_mask'] = supressed_mask
    # print({k:v.shape for k,v in o.items()})
    return o

In [ ]:
{k: v.shape for k,v in ds_a2[0].items() if isinstance(v, torch.Tensor)}


In [ ]:
import gc
import numpy as np

In [ ]:
X = ds_a2['logits']
name = "logits"
score = train_linear_prob_on_dataset(X, name)
results.append((name, score))
score

In [ ]:

X = ds_a2['llm_ans']
y = ds_a2['label']

X_train, y_train = X[:train_test_split], y[:train_test_split]
X_test, y_test = X[train_test_split:], y[train_test_split:]

score = roc_auc(y_test, X_test[:, 0]).item()
results.append(('llm_ans', score))
score

In [ ]:
X = 1-torch.sigmoid(ds_a2['llm_log_prob_true']/10)
y = ds_a2['label']

X_train, y_train = X[:train_test_split], y[:train_test_split]
X_test, y_test = X[train_test_split:], y[train_test_split:]

score = roc_auc(y_test, X_test).item()
results.append(('llm_log_prob_true', score))
score

In [ ]:
# now various eps
for eps in [-50, -10, -5, -1, -0.5, -0.1, -0.01, -0, 0, 0.01, 0.1, 0.5, 1, 10, 50]:
    gc.collect()
    ds_a3 = ds_a2.map(lambda x:calc_hs_sup(x, eps=eps), num_proc=None, batched=True, batch_size=64, desc=f"eps {eps}")
    print(f"eps {eps} ds_a3['supressed_mask'].mean()={ds_a3['supressed_mask'].mean()}")
    data_names = ["supressed_hs", "supressed_mask"]
    for dn in data_names:
        try:
            X = torch.stack([r1f(x) for x in ds_a3[dn]])
            name = f"{dn} {r1} {eps}"
            score = train_linear_prob_on_dataset(X, name)
            results.append((name, score))
        except Exception as e:
            print(f"error with {name}")
            print(e)


In [ ]:
import pandas as pd

# note hs_sup seems to get more important as we lower the thresh
df = pd.DataFrame(results, columns=["name", "auroc"]).sort_values(
    "auroc", ascending=False
)
df

In [ ]:
df['data'] = df['name'].apply(lambda x: x.split()[0])
df2 = df.groupby('data').max().sort_values("auroc", ascending=False)
df2

In [ ]:
df2.index

In [ ]:
# plot it
# TODO add logits

from matplotlib import pyplot as plt
df3 = df2.T[['llm_ans', 'llm_log_prob_true', 'hidden_states',  'supressed_hs', ]].rename(columns={
    'llm_ans': 'LLM Answer',
    'llm_log_prob_true': 'LLM Probability',
    'hidden_states': 'Hidden States',
    'acts': 'Activations: up_proj',
    # 'logits': 'Logits',
    'supressed_hs': 'Supressed Hidden States',
}).T.sort_values("auroc", ascending=False)
df3.plot.barh()
plt.legend().remove()
plt.xlabel(f"Linear probe AUROC")
plt.title(f"TruthfulQA Binary with {model_name}")
plt.xlim(0.5, None)